In [22]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as ski
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
data = pd.read_csv("Data/HEMODB.csv",sep=';')

In [27]:
data.head()

,ID,Brachial SBP,Brachial DBP,Brachial PP,MAP,Heart rate,Carotid-to-femoral PWV,Stroke volume,Ejection fraction,Central SBP,End-systolic elastance,End-diastolic elastance,Ejection duration,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,1,98.328,73.229,25.098,81.595,85.608,6.2060,50.896,48.893,91.399,314000000.0,15600000.0,0.15849,NaN,NaN,NaN
1,2,87.134,64.596,22.538,72.109,74.166,5.8806,51.110,48.303,80.663,269000000.0,18800000.0,0.15855,NaN,NaN,NaN
2,3,85.728,65.301,20.427,72.110,86.010,6.0619,45.413,41.588,79.826,220000000.0,18800000.0,0.16826,NaN,NaN,NaN
3,4,87.308,68.627,18.681,74.854,NaN,5.9547,45.091,39.257,81.771,204000000.0,16500000.0,0.18353,NaN,NaN,NaN
4,5,83.572,65.532,18.040,71.545,97.197,5.7352,41.302,39.403,78.667,219000000.0,17300000.0,0.11478,NaN,NaN,NaN
